In [5]:
!pip install "tensorflow<2.11"

     ---------------------------------------- 0.0/455.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/455.9 MB ? eta -:--:--
     -------------------------------------- 0.0/455.9 MB 388.9 kB/s eta 0:19:33
     -------------------------------------- 0.1/455.9 MB 573.4 kB/s eta 0:13:15
     -------------------------------------- 0.1/455.9 MB 652.2 kB/s eta 0:11:39
     -------------------------------------- 0.1/455.9 MB 652.2 kB/s eta 0:11:39
     -------------------------------------- 0.1/455.9 MB 532.5 kB/s eta 0:14:16
     -------------------------------------- 0.2/455.9 MB 517.2 kB/s eta 0:14:42
     -------------------------------------- 0.2/455.9 MB 597.3 kB/s eta 0:12:43
     -------------------------------------- 0.3/455.9 MB 582.4 kB/s eta 0:13:03
     -------------------------------------- 0.3/455.9 MB 633.2 kB/s eta 0:12:00
     -------------------------------------- 0.4/455.9 MB 674.7 kB/s eta 0:11:16
     -------------------------------------- 0.4/455.9 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf2onnx 1.9.3 requires flatbuffers~=1.12, but you have flatbuffers 24.3.25 which is incompatible.
streamlit 1.24.0 requires protobuf<5,>=3.20, but you have protobuf 3.19.6 which is incompatible.
scipy 1.6.2 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.26.4 which is incompatible.
mlchain 0.3.0 requires numpy<=1.22.4; python_version > "3.7", but you have numpy 1.26.4 which is incompatible.

[notice] A new release of pip is available: 23.0 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
import numpy as np
import pandas as pd 
import random
import cv2

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import keras.backend as keras_be
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.metrics import *


c:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
outputs_folder = "./"
# Define the batch size and image dimensions
BATCH_SIZE = 64
IMAGE_HEIGHT = 256
IMAGE_WIDTH = 256
IMAGE_SIZE = (IMAGE_HEIGHT, IMAGE_WIDTH)
SEED=5005

In [3]:


def JaccardLoss(targets, inputs, smooth=1e-6):
    # Convert targets to float32 data type
    targets = keras_be.cast(targets, dtype='float32')

    # Flatten label and prediction tensors
    inputs = keras_be.flatten(inputs)
    targets = keras_be.flatten(targets)

    # Reshape flattened tensors to rank-2 matrices
    inputs = keras_be.reshape(inputs, (-1, 1))
    targets = keras_be.reshape(targets, (-1, 1))

    # Calculate intersection and union using dot product
    intersection = keras_be.sum(keras_be.dot(keras_be.transpose(targets), inputs))
    total = keras_be.sum(targets) + keras_be.sum(inputs)
    union = total - intersection

    # Calculate IoU (Intersection over Union)
    IoU = (intersection + smooth) / (union + smooth)

    # Calculate Jaccard loss
    return 1.0 - IoU

In [4]:
class PreprocessingBlock(Layer):
    def __init__(self, image_size=IMAGE_SIZE, scale=1./255):
        super().__init__()
        self.resize = Resizing(IMAGE_SIZE[0], IMAGE_SIZE[1])
        self.rescale = Rescaling(scale=1./255)
    def call(self, x):
        x = self.resize(x)
        x = self.rescale(x)
        return x
class AugmentationBlock(layers.Layer):
    def __init__(self, seed=None, flip_mode="horizontal", rotation_factor=0.1):
        super().__init__()
        self.flip = RandomFlip(mode=flip_mode, seed=seed)
        self.rotate = RandomRotation(factor=rotation_factor, fill_mode='nearest', seed=seed)
        
    def call(self, x):
        x = self.flip(x)
        x = self.rotate(x)
        
        return x
class ConvBlock(layers.Layer):
        def __init__(self, filters, kernel_size, strides=1, padding='same', drop_out_rate=0.1, pool=False):
            super().__init__()

            self.conv = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding, kernel_initializer='he_normal')
            self.dropout = Dropout(drop_out_rate)
            self.norm = BatchNormalization()
            self.relu = LeakyReLU(0.1)

        def call(self, x):
            x = self.conv(x)
            x = self.norm(x)
            x = self.relu(x)
            x = self.dropout(x)
            return x


class UpConvBlock(layers.Layer):
    def __init__(self, filters, kernel_size, strides=2, padding='same', drop_out_rate=0.1, use_attention=False):
            super().__init__()
            self.tconv = Conv2DTranspose(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding, kernel_initializer='he_normal')
            self.dropout = Dropout(drop_out_rate)
            self.norm = BatchNormalization()
            self.relu = ReLU()
            self.concat = Concatenate(axis=3)
            self.use_attention = use_attention
            self.attention = AdditiveAttention()
    def call(self, x, y=None):
            x = self.tconv(x)
            if y != None:
                if self.use_attention:
                    x = self.attention([x, y])
                else:
                    x = self.concat([x, y])
            x = self.norm(x)
            x = self.relu(x)
            x = self.dropout(x)
            return x   

In [5]:

#create model
inputs = Input((None, None, 3))

preprocessed = PreprocessingBlock()(inputs)

#     augmented = AugmentationBlock()(preprocessed)
conv1 = ConvBlock(16, 3)(preprocessed)
conv1 = ConvBlock(16, 3)(conv1) 
pool1 = MaxPooling2D((2,2))(conv1)

conv2 = ConvBlock(32, 3)(pool1)
conv2 = ConvBlock(32, 3)(conv2)
pool2 = MaxPooling2D((2,2))(conv2)

conv3 = ConvBlock(64, 3)(pool2)
conv3 = ConvBlock(64, 3)(conv3)
pool3 = MaxPooling2D((2,2))(conv3)

conv4 = ConvBlock(128, 3)(pool3)
conv4 = ConvBlock(128, 3)(conv4)
pool4 = MaxPooling2D((2,2))(conv4)

conv5 = ConvBlock(256, 3)(pool4)
conv5 = ConvBlock(256, 3)(conv5)

upconv6 = UpConvBlock(filters=128, kernel_size=2)(conv5, conv4)
conv6 = ConvBlock(128, 3)(upconv6)
conv6 = ConvBlock(128, 3)(conv6)

upconv7 = UpConvBlock(filters=64, kernel_size=2)(conv6, conv3)
conv7 = ConvBlock(64, 3)(upconv7)
conv7 = ConvBlock(64, 3)(conv7)

upconv8 = UpConvBlock(filters=32, kernel_size=2)(conv7, conv2)
conv8 = ConvBlock(32, 3)(upconv8)
conv8 = ConvBlock(32, 3)(conv8)

upconv9 = UpConvBlock(filters=16, kernel_size=2)(conv8, conv1)
conv9 = ConvBlock(16, 3)(upconv9)
conv9 = ConvBlock(16, 3)(conv9)

outputs = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

# model = tf.keras.Model(inputs=[inputs], outputs=[conv4])
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
metrics = [MeanIoU(num_classes=2)]
model.compile(optimizer='adam', loss=JaccardLoss, metrics=metrics)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 preprocessing_block (Preproces  (None, 256, 256, 3)  0          ['input_1[0][0]']                
 singBlock)                                                                                       
                                                                                                  
 conv_block (ConvBlock)         (None, 256, 256, 16  512         ['preprocessing_block[0][0]']    
                                )                                                             

In [6]:
# !pip install gdown
# !gdown 1kJ0U0VhqqtBPjPbzgZwH5lI3WXquMfUY -O ./lips_model_ep_2.h5

In [7]:
model.load_weights(os.path.join(outputs_folder, 'lips_model_ep_2.h5'))

In [8]:
import cv2
from PIL import Image
import numpy as np
import tensorflow as tf
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import time

# Open the camera
cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")

# Create a colormap
cmp = ListedColormap(['#00000000', 'red'])
i=0

start_time = time.time()  # remember when we started
while (time.time() - start_time) < 10:  # 10 seconds duration
    # Read the image from the camera
    ret, frame = cap.read()

    # Convert the image to PIL format
    image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    image = image.resize(IMAGE_SIZE)

    # Convert the image to numpy array
    image = np.array(image)

    # Expand the dimensions
    image = tf.expand_dims(image, axis=0)

    # Predict the masks
    predict_masks = model.predict(image)

    # Display the original image
    plt.imshow(image[0])

    # Display the mask overlay
    plt.imshow(predict_masks[0], alpha=.5, cmap=cmp)

    # Save the image
    plt.savefig(f"outputs/{i}.jpg")
    plt.close()
    i+=1

# Release the camera
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 106ms/step


: 

In [54]:
cap.release()
cv2.destroyAllWindows()

In [17]:
from PIL import Image
import requests
url = """
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSgLB9wArW13ULvPBirRC9mPi9TY27VTFLsIVvD9DNs8g&s
"""
url=url.strip('\n')
image = Image.open(requests.get(url, stream=True).raw)
image=image.resize(IMAGE_SIZE)
image=np.array(image)
image=tf.expand_dims(image, axis=0)
predict_masks=model.predict(image)

1/1 [==============================] - 0s 116ms/step


In [ ]:
from matplotlib.colors import ListedColormap
cmp=ListedColormap(['#00000000','purple'])
plt.imshow(image[0])
plt.imshow(predict_masks[0], alpha=.5, cmap=cmp)
plt.show()

In [ ]:
def get_cropped_part(image, mask):
    rgb_channels = image[:,:,0], image[:,:,1], image[:,:,2]
    mask_rgb = rgb_channels*mask[:,:,0]
    mask_rgb = np.dstack(mask_rgb)
    return mask_rgb

In [2]:
import tensorflow
tensorflow.__version__

AttributeError: module 'tensorflow' has no attribute '__version__'